In [0]:
rdd1= sc.textFile('dbfs:/FileStore/movies.csv')
rdd2= sc.textFile('dbfs:/FileStore/rating.csv')

In [0]:
print(f"defaultMinPartitions: {sc.defaultMinPartitions}")
print(f"name: rdd1, count: {rdd1.count()}, partitions {rdd1.getNumPartitions()}")
print(f"name: rdd2, count: {rdd1.count()}, partitions {rdd1.getNumPartitions()}")

defaultMinPartitions: 2
name: rdd1, count: 9743, partitions 2
name: rdd2, count: 9743, partitions 2


In [0]:
rdd3 = rdd1.repartition(3) # Switch partitions into 3
def f(iterator): # create Iterator
    ret = yield from iterator # create three iterator since we have tree partitions
    return ret

rdd3.mapPartitions(f).collect()

Out[4]: ['movieId,title,genres',
 '1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 '2,Jumanji (1995),Adventure|Children|Fantasy',
 '3,Grumpier Old Men (1995),Comedy|Romance',
 '4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 '5,Father of the Bride Part II (1995),Comedy',
 '6,Heat (1995),Action|Crime|Thriller',
 '7,Sabrina (1995),Comedy|Romance',
 '8,Tom and Huck (1995),Adventure|Children',
 '9,Sudden Death (1995),Action',
 '30,Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Crime|Drama',
 '31,Dangerous Minds (1995),Drama',
 '32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller',
 '34,Babe (1995),Children|Drama',
 '36,Dead Man Walking (1995),Crime|Drama',
 '38,It Takes Two (1995),Children|Comedy',
 '39,Clueless (1995),Comedy|Romance',
 '40,"Cry, the Beloved Country (1995)",Drama',
 '41,Richard III (1995),Drama|War',
 '42,Dead Presidents (1995),Action|Crime|Drama',
 '66,Lawnmower Man 2: Beyond Cyberspace (1996),Action|Sci-Fi|Thriller',
 '68,Frenc

In [0]:
rdd1 = rdd1.coalesce(1)
print(rdd1.getNumPartitions())

1


In [0]:
rdd3 = rdd1.mapPartitions(lambda it: map(lambda v:v.upper(),it)) # mapPartition allow users iterator, and use one more lambda.
rdd3.take(10)

Out[7]: ['MOVIEID,TITLE,GENRES',
 '1,TOY STORY (1995),ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY',
 '2,JUMANJI (1995),ADVENTURE|CHILDREN|FANTASY',
 '3,GRUMPIER OLD MEN (1995),COMEDY|ROMANCE',
 '4,WAITING TO EXHALE (1995),COMEDY|DRAMA|ROMANCE',
 '5,FATHER OF THE BRIDE PART II (1995),COMEDY',
 '6,HEAT (1995),ACTION|CRIME|THRILLER',
 '7,SABRINA (1995),COMEDY|ROMANCE',
 '8,TOM AND HUCK (1995),ADVENTURE|CHILDREN',
 '9,SUDDEN DEATH (1995),ACTION']

In [0]:
rdd4 = rdd1.mapPartitionsWithIndex(lambda idx, it: map(lambda v: f"idx:{idx}, value:{v}",it)) 
# coalesce integrated partitions into one
# mapPartitionsWithIndex shows partition's index
rdd4.take(10)

Out[13]: ['idx:0, value:movieId,title,genres',
 'idx:0, value:1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy',
 'idx:0, value:2,Jumanji (1995),Adventure|Children|Fantasy',
 'idx:0, value:3,Grumpier Old Men (1995),Comedy|Romance',
 'idx:0, value:4,Waiting to Exhale (1995),Comedy|Drama|Romance',
 'idx:0, value:5,Father of the Bride Part II (1995),Comedy',
 'idx:0, value:6,Heat (1995),Action|Crime|Thriller',
 'idx:0, value:7,Sabrina (1995),Comedy|Romance',
 'idx:0, value:8,Tom and Huck (1995),Adventure|Children',
 'idx:0, value:9,Sudden Death (1995),Action']

In [0]:
df1 = spark.read.csv('dbfs:/FileStore/movies.csv',header= True)
df2 = spark.read.csv('dbfs:/FileStore/ratings.csv',header= True)

In [0]:
join_df = df1.join(df2, ['movieId'], 'inner')


In [0]:
join_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) Project [movieId#168, title#169, genres#170, userId#191, rating#193, timestamp#194]
   +- *(2) BroadcastHashJoin [movieId#168], [movieId#192], Inner, BuildLeft, false, true
      :- ShuffleQueryStage 0, Statistics(sizeInBytes=840.6 KiB, rowCount=9.74E+3, isRuntime=true)
      :  +- Exchange SinglePartition, EXECUTOR_BROADCAST, [plan_id=205]
      :     +- *(1) Filter isnotnull(movieId#168)
      :        +- FileScan csv [movieId#168,title#169,genres#170] Batched: false, DataFilters: [isnotnull(movieId#168)], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/movies.csv], PartitionFilters: [], PushedFilters: [IsNotNull(movieId)], ReadSchema: struct<movieId:string,title:string,genres:string>
      +- *(2) Filter isnotnull(movieId#192)
         +- FileScan csv [userId#191,movieId#192,rating#193,timestamp#194] Batched: false, DataFilters: [isnotnull(movieId#192)], Format: CSV, Location: InMemo